In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, HuberRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import pickle

## Import Data

In [2]:
data = pd.read_csv(r"E:\硕士学习\第二年第二学期\Data Visualization\datasets\datasets\FFG_Hackathon_Country_Level_Data.csv")

In [3]:
data.head(5)

Year Country Name Country Code  AG.LND.AGRI.K2  AG.LND.TOTL.K2  \
0  2007      Albania          ALB        11190.00         27400.0   
1  2007      Algeria          DZA       412520.00       2381740.0   
2  2007       Angola          AGO       508468.17       1246700.0   
3  2007    Argentina          ARG      1440330.00       2736690.0   
4  2007      Armenia          ARM        17519.00         28470.0   

   AG.SRF.TOTL.K2  EG.ELC.COAL.ZS  EG.ELC.FOSL.ZS  EG.ELC.HYRO.ZS  \
0         28750.0        0.000000        2.517483       97.482517   
1       2381740.0        0.000000       99.392408        0.607592   
2       1246700.0        0.000000       22.381100       77.618900   
3       2780400.0        2.179398       63.438998       28.270726   
4         29740.0        0.000000       25.245846       31.417430   

   EG.ELC.NGAS.ZS  ...  EN.ATM.CO2E.SF.ZS  EN.CO2.BLDG.ZS  EN.CO2.ETOT.ZS  \
0        0.000000  ...           1.867414        7.407407        8.730159   
1       97.257770  ...           2.755153       16.028285       40.011786   
2        0.000000  ...           0.000000       23.858921       12.344398   
3       51.046874  ...           2.091227       14.616174       33.816513   
4       25.245846  ...           0.145033       22.291667       22.500000   

   EN.CO2.MANF.ZS  EN.CO2.OTHX.ZS  EN.CO2.TRAN.ZS  EN.POP.DNST  \
0       16.931217       11.640212       55.555556   108.394781   
1       11.455510        0.000000       32.516205    14.345383   
2       23.547718        0.311203       40.041494    16.768559   
3       20.510962        5.405569       25.644742    14.500837   
4       24.375000        8.750000       22.083333   103.007306   

   NY.GDP.TOTL.RT.ZS  SP.RUR.TOTL  SP.URB.TOTL  
0           1.761086    1517619.0    1452398.0  
1          32.464934   11839539.0   22327433.0  
2          46.279012    8879762.0   12025601.0  
3           4.995813    3823185.0   35861110.0  
4           0.009296    1063397.0    1869221.0  

[5 rows x 38 columns]

In [5]:
data = data[["Year", "Country Name", "Country Code","EN.ATM.CO2E.KT", "EG.USE.COMM.CL.ZS", "EG.USE.ELEC.KH.PC", "EG.GDP.PUSE.KO.PP",
             "AG.LND.TOTL.K2", "EG.FEC.RNEW.ZS", "SP.RUR.TOTL", "SP.URB.TOTL"]]

data.head(5)

Year Country Name Country Code  EN.ATM.CO2E.KT  EG.USE.COMM.CL.ZS  \
0  2007      Albania          ALB        3927.357          15.549632   
1  2007      Algeria          DZA      109404.945           0.039704   
2  2007       Angola          AGO       25151.953           1.776957   
3  2007    Argentina          ARG      175176.257           5.774868   
4  2007      Armenia          ARM        5056.793          27.842652   

   EG.USE.ELEC.KH.PC  EG.GDP.PUSE.KO.PP  AG.LND.TOTL.K2  EG.FEC.RNEW.ZS  \
0        1213.124369          10.700006         27400.0       32.100937   
1         894.284691          11.418314       2381740.0        0.411409   
2         133.267239          12.827400       1246700.0       61.599703   
3        2455.656576           8.916315       2736690.0        8.887649   
4        1773.159682           7.762797         28470.0        7.017262   

   SP.RUR.TOTL  SP.URB.TOTL  
0    1517619.0    1452398.0  
1   11839539.0   22327433.0  
2    8879762.0   12025601.0  
3    3823185.0   35861110.0  
4    1063397.0    1869221.0

In [6]:
data.columns = ["Year", "Name", "Country", "CO2 Emission", "Clean Energy", "Electricity" ,"GDP per unit", "Land area", 
                "Renewable energy", "Rural population", "Urban population"]

data.head(5)

Year       Name Country  CO2 Emission  Clean Energy  Electricity  \
0  2007    Albania     ALB      3927.357     15.549632  1213.124369   
1  2007    Algeria     DZA    109404.945      0.039704   894.284691   
2  2007     Angola     AGO     25151.953      1.776957   133.267239   
3  2007  Argentina     ARG    175176.257      5.774868  2455.656576   
4  2007    Armenia     ARM      5056.793     27.842652  1773.159682   

   GDP per unit  Land area  Renewable energy  Rural population  \
0     10.700006    27400.0         32.100937         1517619.0   
1     11.418314  2381740.0          0.411409        11839539.0   
2     12.827400  1246700.0         61.599703         8879762.0   
3      8.916315  2736690.0          8.887649         3823185.0   
4      7.762797    28470.0          7.017262         1063397.0   

   Urban population  
0         1452398.0  
1        22327433.0  
2        12025601.0  
3        35861110.0  
4         1869221.0

In [7]:
gdp = pd.read_csv(r"E:\硕士学习\第二年第二学期\Data Visualization\datasets\datasets\WORLD_GDP.csv")
gdp.drop(columns = ["Country Name","Indicator Name","Indicator Code"], inplace = True)
gdp.head(5)

Country Code          2000          2001          2002          2003  \
0          ABW  1.873184e+09  1.896648e+09  1.962011e+09  2.044134e+09   
1          AFG           NaN           NaN  3.854235e+09  4.539501e+09   
2          AGO  9.129595e+09  8.936064e+09  1.528559e+10  1.781270e+10   
3          ALB  3.480355e+09  3.922101e+09  4.348068e+09  5.611496e+09   
4          AND  1.429049e+09  1.546926e+09  1.755910e+09  2.361727e+09   

           2004          2005          2006          2007          2008  ...  \
0  2.254749e+09  2.359777e+09  2.469832e+09  2.677654e+09  2.843017e+09  ...   
1  5.220824e+09  6.226199e+09  6.971379e+09  9.715762e+09  1.024977e+10  ...   
2  2.355205e+10  3.697092e+10  5.238101e+10  6.526645e+10  8.853861e+10  ...   
3  7.184686e+09  8.052074e+09  8.896073e+09  1.067732e+10  1.288135e+10  ...   
4  2.894922e+09  3.159905e+09  3.456442e+09  3.952601e+09  4.085631e+09  ...   

           2012          2013          2014          2015          2016  \
0  2.615084e+09  2.727933e+09  2.791061e+09  2.963128e+09  2.983799e+09   
1  2.020357e+10  2.056449e+10  2.055058e+10  1.999816e+10  1.801956e+10   
2  1.250000e+11  1.330000e+11  1.370000e+11  8.721930e+10  4.984049e+10   
3  1.231983e+10  1.277622e+10  1.322815e+10  1.138685e+10  1.186120e+10   
4  3.188809e+09  3.193704e+09  3.271808e+09  2.789870e+09  2.896679e+09   

           2017          2018          2019          2020          2021  
0  3.092179e+09  3.202235e+09  3.368970e+09  2.610039e+09  3.126019e+09  
1  1.889635e+10  1.841885e+10  1.890449e+10  2.014344e+10  1.478686e+10  
2  6.897277e+10  7.779294e+10  6.930911e+10  5.361907e+10  6.740429e+10  
3  1.301969e+10  1.515643e+10  1.540183e+10  1.513187e+10  1.825579e+10  
4  3.000181e+09  3.218316e+09  3.155065e+09  2.891022e+09  3.330282e+09  

[5 rows x 23 columns]

In [8]:
melted_gdp = gdp.melt(id_vars=['Country Code'], var_name='Year', value_name='GDP')
melted_gdp = melted_gdp[["Year","Country Code","GDP"]]
melted_gdp.columns = ["Year","Country","GDP"]
melted_gdp.head(5)

Year Country           GDP
0  2000     ABW  1.873184e+09
1  2000     AFG           NaN
2  2000     AGO  9.129595e+09
3  2000     ALB  3.480355e+09
4  2000     AND  1.429049e+09

In [9]:
country = np.unique(gdp["Country Code"])
country_dict = dict(zip(list(country), range(len(country))))
data["Country"] = data["Country"].apply(lambda x: country_dict[x])
melted_gdp["Country"] = melted_gdp["Country"].apply(lambda x: country_dict[x])
melted_gdp["Year"] = melted_gdp["Year"].astype(int)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_41416\1473676495.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Country"] = data["Country"].apply(lambda x: country_dict[x])


In [10]:
melted_gdp.iloc[0,0]

2000

In [11]:
data = data.merge(melted_gdp, how = "left", on = ["Year", "Country"])

In [12]:
data = data.loc[data["Year"]!=2016,:]
data.fillna(method = "ffill", inplace = True)

In [13]:
data["Population"] = data["Rural population"]+data["Urban population"]
data["Urbanization"] = data["Urban population"]/data["Population"]

In [14]:
data.columns

Index(['Year', 'Name', 'Country', 'CO2 Emission', 'Clean Energy',
       'Electricity', 'GDP per unit', 'Land area', 'Renewable energy',
       'Rural population', 'Urban population', 'GDP', 'Population',
       'Urbanization'],
      dtype='object')

In [15]:
data = data[['Year', 'Clean Energy', 'Electricity', 'Land area', 'Renewable energy', 
             'GDP', 'Population', 'Urbanization', 'CO2 Emission','Name']]

data.head(5)

Year  Clean Energy  Electricity  Land area  Renewable energy           GDP  \
0  2007     15.549632  1213.124369    27400.0         32.100937  1.067732e+10   
1  2007      0.039704   894.284691  2381740.0          0.411409  1.350000e+11   
2  2007      1.776957   133.267239  1246700.0         61.599703  6.526645e+10   
3  2007      5.774868  2455.656576  2736690.0          8.887649  2.880000e+11   
4  2007     27.842652  1773.159682    28470.0          7.017262  9.206302e+09   

   Population  Urbanization  CO2 Emission       Name  
0   2970017.0       0.48902      3927.357    Albania  
1  34166972.0       0.65348    109404.945    Algeria  
2  20905363.0       0.57524     25151.953     Angola  
3  39684295.0       0.90366    175176.257  Argentina  
4   2932618.0       0.63739      5056.793    Armenia

In [50]:
X = data[['Year','Clean Energy', 'Electricity', 'Land area', 'Renewable energy',
          'GDP', 'Population', 'Urbanization']]
y = data["CO2 Emission"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 35)

## Try several models

In [51]:
linear = LinearRegression()
linear.fit(X_train, y_train)
y_pred = linear.predict(X_test)
y_estm = linear.predict(X_train)
print("train mse", mean_squared_error(y_estm, y_train))
print("test mse", mean_squared_error(y_pred, y_test))
print("train r2", r2_score(y_estm, y_train))
print("test r2", r2_score(y_pred, y_test))

train mse 124732563504.78323
test mse 148275338888.5984
train r2 0.8506043336372392
test r2 0.7482029352105597


In [52]:
rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)
y_estm = rfr.predict(X_train)
print("train mse", mean_squared_error(y_estm, y_train))
print("test mse", mean_squared_error(y_pred, y_test))
print("train r2", r2_score(y_estm, y_train))
print("test r2", r2_score(y_pred, y_test))

train mse 2396220295.865191
test mse 11419401791.331171
train r2 0.9973975476809722
test r2 0.9744075200713876


In [29]:
# write the model into pickle file
with open('E:/硕士学习/第二年第二学期/Data Visualization/Assignment3/rf_model.pkl', 'wb') as f:
    
    pickle.dump(rfr, f)

In [30]:
# test read the model
with open('E:/硕士学习/第二年第二学期/Data Visualization/Assignment3/rf_model.pkl', 'rb') as f:
    
    rfr_load = pickle.load(f)

y_pred = rfr_load.predict(X_test)

In [16]:
# write the data into a csv
data.to_csv("E:/硕士学习/第二年第二学期/Data Visualization/Assignment3/total_data.csv", index = None)